In [1]:
#### Python 3
#### Author : Vibhor Kashyap
#### vibhor_kashyap@optum.com

In [2]:
import re
import csv
import json
import random
from googlesearch import search 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from tqdm import tqdm

#### read the excel file which mentions suspicious phone numbers

In [3]:
df = pd.read_excel(r'C:\Users\vkashya5\Desktop\Phone number fraud list v2.xlsx',header=None)
numberlist = list(df[0])

#### web agents to be used in scraping

In [4]:
USER_AGENTS = [
    ('Mozilla/5.0 (X11; Linux x86_64) '
     'AppleWebKit/537.36 (KHTML, like Gecko) '
     'Chrome/57.0.2987.110 '
     'Safari/537.36'),  # chrome
    ('Mozilla/5.0 (X11; Linux x86_64) '
     'AppleWebKit/537.36 (KHTML, like Gecko) '
     'Chrome/61.0.3163.79 '
     'Safari/537.36'),  # chrome
    ('Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) '
     'Gecko/20100101 '
     'Firefox/55.0'),  # firefox
    ('Mozilla/5.0 (X11; Linux x86_64) '
     'AppleWebKit/537.36 (KHTML, like Gecko) '
     'Chrome/61.0.3163.91 '
     'Safari/537.36'),  # chrome
    ('Mozilla/5.0 (X11; Linux x86_64) '
     'AppleWebKit/537.36 (KHTML, like Gecko) '
     'Chrome/62.0.3202.89 '
     'Safari/537.36'),  # chrome
    ('Mozilla/5.0 (X11; Linux x86_64) '
     'AppleWebKit/537.36 (KHTML, like Gecko) '
     'Chrome/63.0.3239.108 '
     'Safari/537.36'),  # chrome
]

#### Scrape google results (urls) using phone numbers as query string. I save the jsons subsequently because there was a risk of google 502'ing the IP address. the pause parameter in search routine is a delay, 70 is a safe limit that didnt get 502'd

In [ ]:
search_list = []
counter = 1
for i in tqdm(numberlist):
    phone = str(i)
    query = str(phone)
    search_results = []
    
    user_agent = random.choice(USER_AGENTS)
    
    for result in search(query, tld="com", num=10, stop=10, pause=70,user_agent = user_agent): 
            search_results.append(result)
            
    d = {}
    d['phone'] = phone
    d['google_search'] = search_results
    search_list.append(d)
    #the location to save successive json files. I kept saving successive jsons to avoid information loss as 
    #was a risk of getting 502'ed by Google
    savedir = r'C:\Users\vkashya5\Desktop\PHONE_NUMBER_SCRAPE\json_download\google_scrape'
    if not os.path.exists(savedir):
        os.makedirs(savedir)
    savepath = os.path.join(savedir,'json_'+str(counter)+'.json')
    counter += 1
    
    with open(savepath,'w') as outfile:
        json.dump (search_list,outfile)
    
        

#### After the above step gets completed, we are left with a final json file. This block identifies the last saved json.

In [5]:
import os
json_save_path = r'C:\Users\vkashya5\Desktop\PHONE_NUMBER_SCRAPE\json_download\google_scrape'
latestfilenum = 0
filenamekeyword = ""
for filename in os.listdir(json_save_path):
    filename_split = filename.split('.')[0]
    filename_split_1 = filename_split.split('_')[1]
    filenamekeyword = filename_split.split('_')[0]
    filename_split_2 = filename_split_1.split('.')[0]
    if int(latestfilenum) < int(filename_split_2):
        latestfilenum = filename_split_2
    
latestfilename = str(filenamekeyword)+"_"+str(latestfilenum)+".json"

#cleanup of unneccesary files
for filename in os.listdir(json_save_path):
    if filename == latestfilename:
        continue
    else:
        os.remove(os.path.join(json_save_path,filename))
        
import_path = os.path.join(json_save_path,latestfilename)

#### An analysis over all the identified links yielded a result that only these urls are relevant for finding addresses

In [25]:
useful_urls = ['opennpi',
 'millionphone',
 'daliya-dance',
 'e-physician',
 'hipaaspace',
 'whitepages',
 'callername',
 'npino',
 'manta',
 'zmaps']

#### scrape and save the html content of relevant urls

In [26]:
address_save_dir = r'C:\Users\vkashya5\Desktop\PHONE_NUMBER_SCRAPE\json_download'      

In [29]:
with open(import_path,'r') as jsonfile:
    data = json.load(jsonfile)
    
analyse_ = []

for i in tqdm(data):
    for url in i['google_search']:
        for sane in useful_urls:
            d = {}
            d['google_search'] = []
            if re.findall(sane, url):
                try:
                    page = requests.get(url)
                    soup = BeautifulSoup(page.text, 'html.parser')

                    d['google_search'].append(url)
                    d['soup'] = str(soup)
                    d['phone'] = i['phone']
                    analyse_.append(d)
                except:
                    continue
              
            else:
                pass




  0%|                                                                                          | 0/127 [00:00<?, ?it/s]

  2%|█▎                                                                                | 2/127 [00:08<09:19,  4.48s/it]

  2%|█▉                                                                                | 3/127 [00:11<08:20,  4.04s/it]

  3%|██▌                                                                               | 4/127 [00:13<06:57,  3.39s/it]

  4%|███▏                                                                              | 5/127 [00:14<05:05,  2.50s/it]

  5%|███▊                                                                              | 6/127 [00:22<08:28,  4.20s/it]

  6%|████▌                                                                             | 7/127 [00:23<06:34,  3.28s/it]

  6%|█████▏                                                                            | 8/127 [00:32<09:38,  4.86s/it]

  7%|█████▊                   

In [30]:
savepath = os.path.join(address_save_dir,'address_fraud.json')
with open(savepath,'w+') as outfile:
    json.dump (analyse_,outfile)

In [31]:
with open(os.path.join(address_save_dir,'address_fraud.json'),'r') as infile:
    relevant_data = json.load(infile)

#### scrape the relevant urls for addresses

In [78]:
scraped_results =  []
for i in relevant_data:
    d = {}
    d['phone'] = i['phone']
    d['addresses'] = []
    d['source'] = ""
    for url in i['google_search']:
        if 'opennpi' in url:
            try:
                page = requests.get(url)
                soup = BeautifulSoup(page.text, 'html.parser')
                soup_str= str(soup)
                start = soup_str.find("<strong>Address:</strong>")
                end = soup_str.find("<br/><strong>Phone:</strong>")
                address = soup_str[start+len("<strong>Address:</strong>"):end]
                d['addresses'].append([address])
                d['source'] = 'opennpi'
            except:
                pass
            
        if 'http://www.e-physician.info/NPI-' in url:
            try:
                page = requests.get(url)
                soup = BeautifulSoup(page.text, 'html.parser')
                soup_str= str(soup)
                start = soup_str.find("Practice")
                end = soup_str.find("Business")
                address = soup_str[start+len("Practice Location: "):end]
                address = address.replace('<div class="indent">',"").split('<br/>')[0]
                d['addresses'].append([str(address.replace('\xa0'," "))])
                d['source']  = 'e-physician'
            except:
                pass
            
#         elif 'millionphone' in url:
#             try:
#                 page = requests.get(url)
#                 soup = BeautifulSoup(page.text, 'html.parser')
#                 soup_str= str(soup)
#                 start = soup_str.find("The call comes from")
#                 end = soup_str.find("Here you can see")
#                 address = soup_str[start+len("The call comes from "):end]
#                 address = address.replace(".", "")
#                 d['addresses'].append([address])
#                 d['source']  = 'millionphone'
#             except:
#                 pass
            
        elif 'hipaaspace' in url:
            if 'national_provider_identifier' in url:
                npi = url[url.index('npi_')+4::].replace(".aspx","")
                try:
                    url = 'http://www.hipaaspace.com/medical_billing/coding/national_provider_identifier/codes/npi_'+npi+'.json'
                    page = requests.get(url)
                    soup = BeautifulSoup(page.text, 'html.parser')
                    js = soup.find(class_="language-json")
                    j = json.loads(js.text)
                    newdict = {}
                    address = j['Npi']['FirstLineMailingAddress']+" "
                    if j['Npi']['SecondLineMailingAddress']:
                        address+=(j['Npi']['SecondLineMailingAddress']+" ")
                    if j['Npi']['MailingAddressCityName']:
                        address+=(j['Npi']['MailingAddressStateName']+" ")
                    d['addresses'].append([address])
                    d['source']  = 'hipaaspace'
                except:
                    pass
                
#         elif 'callername' in url:
#             try:
#                 page = requests.get(url)
#                 soup = BeautifulSoup(page.text, 'html.parser')
#                 mydivs = soup.findAll("div", {"class": "callerid"})
#                 if mydivs[0]:
#                     start = str(mydivs[0]).find('<p>')
#                     end = str(mydivs[0]).find('</p>')
#                     addstring = str(mydivs[0])
#                     address = addstring[start+len('<p>'):end]
#                     d['addresses'].append([address])
#                     d['source']  = 'callername'
                    

#             except Exception as e:
#                 error = e
#                 page = requests.get(url)
#                 soup = BeautifulSoup(page.text, 'html.parser')
#                 mydivs = soup.findAll("div", {"class": "panel-footer"})
#                 start = str(mydivs[1]).find("<strong>")
#                 end = str(mydivs[1]).find("</strong>")
#                 address = str(mydivs[1])[start+len("<strong>"):end]
#                 d['addresses'].append([address])
#                 d['source']  = 'callername'
        
        elif 'npino' in url:
            try:
                page = requests.get(url)
                soup = BeautifulSoup(page.text, 'html.parser')
                table = soup.find_all("table",{"class":"table table-striped table-condensed"})
                row_data = []
                for row in table[0].find_all("tr"):
                    cols = row.find_all("td")
                    cols = [ele.text.strip() for ele in cols]
                    row_data.append(cols)

                for row in table[1].find_all("tr"):
                    cols = row.find_all("td")
                    cols = [ele.text.strip() for ele in cols]
                    row_data.append(cols)

                for row in table[2].find_all("tr"):
                    cols = row.find_all("td")
                    cols = [ele.text.strip() for ele in cols]
                    row_data.append(cols)
                address=""
                for i in row_data:
                    if i[0]=='Address':
                        for x in i[1:]:
                            address+=x
                        break
                d['addresses'].append([address])
                d['source']  = 'npino'
            except:
                pass
            
        elif 'zmaps' in url:
            try:
                page = requests.get(url)
                soup = BeautifulSoup(page.text, 'html.parser')

                start=str(soup).find('<address>')
                end=str(soup).find('</address>')
                
                address= str(soup)[start+len('<address>'):end]
                address=address.replace("<h3>","")
                address=address.replace("</h3>","")
                address=address.replace("\n","")
                d['addresses'].append([address])
                d['source']  = 'zmaps'
            except:
                pass
        scraped_results.append(d)   
               

In [79]:
clean_scraped_results = [x for x in scraped_results if x['addresses']]

In [80]:
for i in clean_scraped_results:
    print(i)

{'phone': '5645409777', 'addresses': [[' 2121 10th Ave N, Lake Worth, FL 33461-3345']], 'source': 'opennpi'}
{'phone': '5615409777', 'addresses': [['2121 10TH AVE N FL ']], 'source': 'hipaaspace'}
{'phone': '5615409777', 'addresses': [['2121 10TH AVE N FL ']], 'source': 'hipaaspace'}
{'phone': '9549900967', 'addresses': [[' 4400 N Federal Hwy Ste 48, Boca Raton, FL 33431-3426']], 'source': 'opennpi'}
{'phone': '9549900967', 'addresses': [[' 6241 N Dixie Hwy Fl 2, Fort Lauderdale, FL 33334-3620']], 'source': 'opennpi'}
{'phone': '7867033808', 'addresses': [['7392 Nw 35th Ter Ste 305, ,MiamiFlorida, 33122-1260']], 'source': 'npino'}
{'phone': '7867033808', 'addresses': [['7392 Nw 35th Ter Ste 305, ,MiamiFlorida, 33122-1260']], 'source': 'npino'}
{'phone': '7867033808', 'addresses': [['7392 NW 35 TERRA SUITE 305 FL ']], 'source': 'hipaaspace'}
{'phone': '3052547576', 'addresses': [['11255 SW 211TH ST SUITE 2 FL ']], 'source': 'hipaaspace'}
{'phone': '3052547576', 'addresses': [['11255 SW 

In [68]:
remove_duplicates = []
for i in clean_scraped_results_:
    for j in clean_scraped_results_:
        if i['phone'] == j['phone']:
            if len(i['addresses']) > len(j['addresses']):
                remove_duplicates.append(i)
        
                        
            

In [70]:
clean_scraped_results

[{'phone': '5645409777',
  'addresses': [[' 2121 10th Ave N, Lake Worth, FL 33461-3345']],
  'source': 'opennpi'},
 {'phone': '5615409777',
  'addresses': [['2121 10TH AVE N FL ']],
  'source': 'hipaaspace'},
 {'phone': '5615409777',
  'addresses': [['2121 10TH AVE N FL ']],
  'source': 'hipaaspace'},
 {'phone': '9549900967',
  'addresses': [[' 4400 N Federal Hwy Ste 48, Boca Raton, FL 33431-3426']],
  'source': 'opennpi'},
 {'phone': '9549900967',
  'addresses': [[' 6241 N Dixie Hwy Fl 2, Fort Lauderdale, FL 33334-3620']],
  'source': 'opennpi'},
 {'phone': '7867033808',
  'addresses': [['7392 Nw 35th Ter Ste 305, ,MiamiFlorida, 33122-1260']],
  'source': 'npino'},
 {'phone': '7867033808',
  'addresses': [['7392 Nw 35th Ter Ste 305, ,MiamiFlorida, 33122-1260']],
  'source': 'npino'},
 {'phone': '7867033808',
  'addresses': [['7392 NW 35 TERRA SUITE 305 FL ']],
  'source': 'hipaaspace'},
 {'phone': '3052547576',
  'addresses': [['11255 SW 211TH ST SUITE 2 FL ']],
  'source': 'hipaaspac

In [51]:
with open(os.path.join(address_save_dir,'address_scrjson'),'w') as outfile:
    json.dump(clean_scraped_results, outfile)   

In [52]:
clean_scraped_results_=[]
for i in clean_scraped_results:
    d = {}
    d['phone'] = i['phone']
    d['source'] = i['source']
    for index,j in enumerate(i['addresses']):
        d['addresses'] = i['addresses'][index][0].strip()
        clean_scraped_results_.append(d)    

In [53]:
df = pd.DataFrame(clean_scraped_results_)

In [55]:
df.to_excel(r'C:\Users\vkashya5\Desktop\PHONE_NUMBER_SCRAPE\meeting.xlsx'  )